In [74]:
import pandas as pd
#from pandas.io.parsers import ParserError
import numpy as np
from helper import get_mapper
import json
import os
import re

In [36]:
from os import listdir, stat
from os.path import isfile, join
BASE_DIR = "."
MIN_SIZE = 512

In [37]:
def extract_blockidf(fullname):
    return fullname.split("Generation_DE ")[1].rsplit('[MW]')[0].strip()

def get_smard_name(f):
    return f.rsplit("/")[2].rsplit("_", 3)[0].strip()

In [38]:
def get_files_from_folder(folder):
    onlyfiles = [folder + "/" + f for f in listdir(folder) if isfile(join(folder, f))]
    onlyfiles.sort()
    files = [f for f in onlyfiles if stat(f).st_size > MIN_SIZE]
    return files

In [39]:
def convert2plantid(df, plantname):
    oldcols = list(df.columns)
    newcols = [extract_blockidf(x) for x in list(df.columns)[1:]]
    newcols2 = ['produced_at'] + [mapper[plantname][x] for x in newcols]
    test = dict(zip(oldcols, newcols2))
    result = df.rename(columns=test)
    #print(oldcols)
    #print(newcols2)
    return result

In [40]:
def get_plant_from_prod_name(prodname):
    tmp = ""
    try:
        tmp = mapper[prodname]
    except KeyError:
        print("plant " + prodname + " not found!")
        return None, None
    blocklist = tmp['list']
    block = blocklist[0]
    blockid = tmp[block]
    
    try:
        plantidx = bpm.loc[bpm.blockid == blockid, 'plantid'].item()
    except ValueError:
        return blockid, np.nan
    
    return blockid, plantidx

In [41]:
bpm = pd.read_csv("../basic/block_plant_mapper.csv")
bpm['plantid'] = bpm['plantid'].apply(lambda x: str(x).replace('/', '_'))

In [42]:
FOLDERS = [os.path.join(BASE_DIR, o) for o in os.listdir(BASE_DIR) if os.path.isdir(os.path.join(BASE_DIR,o))]
FOLDERS.sort()
FOLDERS = FOLDERS[1:-3]

In [43]:
FOLDERS

['./2015', './2016', './2017', './2018', './2019', './2020', './2021']

In [44]:
FILES_L = [get_files_from_folder(f) for f in FOLDERS]
FILES = [item for sublist in FILES_L for item in sublist]

In [45]:
mapper = get_mapper('../production/plantmapper.json')

In [46]:
#mapper

In [47]:
#bpm

In [48]:
#FILES

In [49]:
def get_year(fn):
    return int(fn.split("/")[1])

In [50]:
prod_mapper = []
for f in FILES:
    fn = get_smard_name(f)
    #print(fn)
    blockid, plantid = get_plant_from_prod_name(fn)
    year = 2015
    try:
        year = get_year(f)
    except IndexError:
        print(fn)
    prod_mapper.append([f, blockid, plantid, year])

plant Kraftwerk_K_htai not found!
plant Pumpspeicherkraftwerk_Vianden not found!
plant Wilhelmshaven not found!
plant Kraftwerk_K_htai not found!
plant Pumpspeicherkraftwerk_Vianden not found!
plant Wilhelmshaven not found!
plant Kraftwerk_K_htai not found!
plant Pumpspeicherkraftwerk_Vianden not found!
plant Wilhelmshaven not found!
plant Kraftwerk_K_htai not found!
plant Pumpspeicherkraftwerk_Vianden not found!
plant Wilhelmshaven not found!
plant Kraftwerk_K_htai not found!
plant Pumpspeicherkraftwerk_Vianden not found!
plant Wilhelmshaven not found!
plant Abwinden-Asten not found!
plant Altenw_rth not found!
plant Donaustadt not found!
plant Fernheizkraftwerk_Mellach not found!
plant Gerlos not found!
plant Greifenstein not found!
plant Gro_kraftwerk_Mannheim not found!
plant Gro_kraftwerk_Mannheim not found!
plant Gro_kraftwerk_Mannheim not found!
plant GuD_Marzahn not found!
plant H_usling not found!
plant Kaprun_Hauptstufe not found!
plant Kaprun_Limberg_II not found!
plant Kapr

In [51]:
get_plant_from_prod_name("Buschhaus")

('BNA0439', '03-01-01012110180')

In [52]:
get_year("./2015/Boxberg_201501010000_201512312345_71.csv")

2015

In [96]:
df = pd.read_csv("./2021/Isar_2_202101010000_202112312359_68.csv", delimiter=";", na_values=['-'], thousands='.', decimal=',', parse_dates=[["Datum", "Uhrzeit"]], on_bad_lines='skip')

In [97]:
df.fillna(0, inplace=True)
df[df.columns[1:]] = df[df.columns[1:]].astype(int)

In [99]:
df[df.columns[1:]] = df[df.columns[1:]].astype(int)

In [101]:
df

,Datum_Uhrzeit,Generation_DE KKI 2[MW]
0,2021-01-01 00:00:00,1423
1,2021-01-01 01:00:00,1424
2,2021-01-01 02:00:00,1423
3,2021-01-01 03:00:00,1423
4,2021-01-01 04:00:00,1428
...,...,...
8755,2021-12-31 19:00:00,1358
8756,2021-12-31 20:00:00,1244
8757,2021-12-31 21:00:00,1064
8758,2021-12-31 22:00:00,1108


In [93]:
df.sort_values(by='Generation_DE KKI 2[MW]')

,Datum_Uhrzeit,Generation_DE KKI 2[MW]
6565,2021-01-10 14:00:00,0.0
6631,2021-04-10 08:00:00,0.0
6630,2021-04-10 07:00:00,0.0
6629,2021-04-10 06:00:00,0.0
6628,2021-04-10 05:00:00,0.0
...,...,...
1878,2021-03-20 06:00:00,1436.0
1883,2021-03-20 11:00:00,1437.0
941,2021-09-02 05:00:00,1437.0
950,2021-09-02 14:00:00,1437.0


In [53]:
#prod_mapper 

In [54]:
prd_df = pd.DataFrame(prod_mapper, columns = ['file', 'blockid', 'plantid', 'year'])

In [55]:
#prd_df

In [104]:
for index, row in list(prd_df.iterrows()):
    dfname = row[0]
    plantid = str(row[2])
    year = str(int(row[3]))
    smardname = get_smard_name(dfname)
    try:
        df = pd.read_csv(dfname, delimiter=";", na_values=['-'], thousands='.', decimal=',', parse_dates=[["Datum", "Uhrzeit"]], on_bad_lines='skip')
    except ValueError:
        df = pd.read_csv(dfname, delimiter=";", na_values=['-'], thousands='.', decimal=',', parse_dates=[["Date", "Time of day"]], on_bad_lines='skip')
    try:
        newdf = convert2plantid(df, smardname)
    except (IndexError, KeyError):
        print(smardname)
        continue
    newdf.fillna(0, inplace=True)
    newdf[newdf.columns[1:]] = newdf[newdf.columns[1:]].astype(int)
    #try:
    #print(dfname, year)
    #print(type(year))
    newdf.to_csv("./by_plantid/" + year + "/" + plantid + '.csv', index=False)
    #except TypeError:
    #    print(year, smardname)
    #print(newdf.dtypes)

Heizkraftwerk_Dresden-Nossener_Br_cke
Kraftwerk_BASF_Ludwigshafen_Mitte
Kraftwerk_K_htai
Pumpspeicherkraftwerk_Vianden
Wilhelmshaven
Heizkraftwerk_Dresden-Nossener_Br_cke
Kraftwerk_BASF_Ludwigshafen_Mitte
Kraftwerk_K_htai
Pumpspeicherkraftwerk_Vianden
Wilhelmshaven
Heizkraftwerk_Dresden-Nossener_Br_cke
Kraftwerk_BASF_Ludwigshafen_Mitte
Kraftwerk_K_htai
Pumpspeicherkraftwerk_Vianden
Wilhelmshaven
Heizkraftwerk_Dresden-Nossener_Br_cke
Kraftwerk_BASF_Ludwigshafen_Mitte
Kraftwerk_K_htai
Pumpspeicherkraftwerk_Vianden
Wilhelmshaven
Heizkraftwerk_Dresden-Nossener_Br_cke
Kraftwerk_BASF_Ludwigshafen_Mitte
Kraftwerk_K_htai
Pumpspeicherkraftwerk_Vianden
Wilhelmshaven
Abwinden-Asten
Altenw_rth
Donaustadt
Fernheizkraftwerk_Mellach
Gerlos
Greifenstein
Gro_kraftwerk_Mannheim
Gro_kraftwerk_Mannheim
Gro_kraftwerk_Mannheim
GuD_Marzahn
H_usling
Heizkraftwerk_Dresden-Nossener_Br_cke
Kaprun_Hauptstufe
Kaprun_Limberg_II
Kaprun_Oberstufe
Kraftwerk_BASF_Ludwigshafen_Mitte
Kraftwerk_K_htai
Laufenburg
Linz-S_d_

In [105]:
extract_blockidf('Generation_DE Q[MW]')

'Q'